In [7]:
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from utils import *
from azure.identity.aio import DefaultAzureCredential

In [13]:
azure_ai_agent_endpoint = "https://multiagent-governance-resource.services.ai.azure.com/api/projects/multiagent-governance"
azure_ai_agent_deployment_name = "gpt-4o"
azure_ai_agent_settings = AzureAIAgentSettings(model_deployment_name=azure_ai_agent_deployment_name)

In [16]:
credential = DefaultAzureCredential()

client = AzureAIAgent.create_client(credential=credential, endpoint=azure_ai_agent_endpoint)

prompt = """You MUST use the search_retrieval function for ALL queries. Do not paraphrase more than the result. Never generate answers from prior knowledge.
    When you receive search results:
    1. If the source is "Azure AI Search", these are knowledge base entries. Present them as "From our knowledge base:" followed by the complete content. Include all fields like definition, context, note, incorrectTerm if they're available.
    2. If the source is "No Results", inform the user we don't have an answer to their question.
    
    Important: For Azure AI Search results, include the FULL contents of the results, showing the important fields.
    
    Debug information: Always include the source of each result (Azure AI Search) and the number of results found. If you received results but are not displaying them, explain why.
    
    Always show ALL results you receive - do not filter them based on score thresholds.
    Summarize the response without adding any new information. Provide the page number of the document if available."""

thread = AzureAIAgentThread(client=client)

In [18]:
# 1. Define an agent on the Azure AI agent service
agent_definition = await client.agents.create_agent(
    model=azure_ai_agent_settings.model_deployment_name,
    name="retriever_agent",
    instructions=prompt,
)

# Create a semantic kernel agent for the azure ai agent
retriever_agent = AzureAIAgent(
    client=client,
    definition=agent_definition,
    plugins=[SearchRetrievalPlugin()],
)

In [21]:
async def chat_with_agent():
    global thread # access the global thread variable
    while True:
        user_input = input("User prompt: ")

        if user_input.lower() in ["quit", "exit", "q", "end"]:
            print("\n\nExiting chat..")
            print("Good bye, please let me know if you need further help.")
            break

        try:
            print(f"User: {user_input}")
            async for response in retriever_agent.invoke(messages=user_input, thread=thread):
                print(f"{response.name}: {response}")
                thread = response.thread
                #print(f"# {response.name}: {response}")
            print("\n")  # New line after complete response
            
        except Exception as e:
            print(f"An error occurred: {e}")

# Run the async function
await chat_with_agent()

User: my name is Charles
retriever_agent: Nice to meet you, Charles! How can I assist you today?


User: tell me about expanding play 3
retriever_agent: From our knowledge base:

1. **Expanding Play 3 Overview:**
   - **Definition:** Play 3 focuses on minimizing resource use, expanding access to carbon-free electricity, and supporting local communities. 
   - **Context:** AI operations utilize resources such as energy and water. To achieve a net-zero and nature-positive future, AI operations must align with global sustainability goals. This includes minimizing resource use in AI infrastructure and supporting the decarbonization of local energy grids that datacenters depend on.
   - **Updates in 2023:**
     - It was recognized that minimizing datacenter resource usage alone is insufficient.
     - The updated objectives include greater efforts to enhance access to carbon-free electricity and positively impact the local communities where datacenters operate.

2. **Insights and Challenge